In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [2]:
faults = dd.read_csv('../data/faults.csv', dtype={'EquipmentID': 'object'})
diagnostics = dd.read_csv('../data/vehicle_onboard_diagnostic_data.csv')

In [3]:
test_df = faults.head(10)
test_df.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [4]:
# applying some geovalues to the faults dataset

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent=f'the_index_out_of_rangers')

def get_location_info(row):
    lat = '38.857638' # row['Latitude']
    long = '-84.626851' # row['Longitude']
    location = geolocator.reverse(f'{lat},{long}')
    address = location.raw['address']
    address['place_id'] = location.raw['place_id']
    return address

applied_df = faults.apply(get_location_info, axis='columns', result_type='expand', meta={'tourism': 'object', 'road': 'object', 'town': 'object', 'county': 'object', 'state': 'object', 'postcode': 'object', 'country': 'object', 'country_code': 'object', 'place_id': 'int64'})
faults = dd.concat([faults, applied_df], axis='columns', ignore_unknown_divisions=True)

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='the_index_out_of_rangers')
location = geolocator.reverse(f'{35.586851},{-86.444583}')
print(location.raw['place_id'])

103663378
